In [31]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


In [32]:
from dotenv import load_dotenv
load_dotenv()

True

In [33]:
import pandas as pd
books = pd.read_csv("books_cleaned1.csv")

In [34]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web:A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5084,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5085,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5086,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that:Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [35]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5083    9788172235222 On A Train Journey Home To North...
5084    9788173031014 This book tells the tale of a ma...
5085    9788179921623 Wisdom to Create a Life of Passi...
5086    9788185300535 This collection of the timeless ...
5087    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5088, dtype: object

In [36]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index = False,
                                   header = False
                                   )

In [37]:
from langchain_core.documents import Document
documents = []
with open("tagged_description.txt","r",encoding = "utf-8") as f:
    for line in f:
        line_content = line.strip()
        if line_content:
            documents.append(Document(page_content = line_content))

print(f"Total Documents created:{len(documents)}")
print("--Document 1--")
print(documents[0].page_content)


Total Documents created:5088
--Document 1--
9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebrat

In [42]:
db_books.delete_collection()

In [43]:
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings()
)

In [44]:
query = "A book to reach children about nature"
docs = db_books.similarity_search(query , k = 3)
docs

[Document(id='0cbfce3e-a194-436c-9a77-2e2582e00977', metadata={}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='c47e2135-b2bc-44c5-a352-6ca15e6e931d', metadata={}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='f1b23af1-006c-4b27-a277-d60ce80cfaed', metadata={}, page_content="9780786808397 Introduce your baby to birds, cats, dogs, and ba

In [45]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())] #extracting isbn13


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3664,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [48]:
def retrieve_semantic_recommendations(query : str, top_k : int = 10)-> pd.DataFrame:
    recs = db_books.similarity_search(query , k = 10)

    books_list=[]

    for i in range(0,len(recs)):
        books_list += [int(recs[i].page_content.strip('""').split()[0])]

    return books[books["isbn13"].isin(books_list)]    






In [49]:
retrieve_semantic_recommendations("A book about magical world")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
354,9780061094156,0061094153,Imajica II,Clive Barker,Fiction,http://books.google.com/books/content?id=DZVKS...,The magical tale of ill-fated lovers lost amon...,1995.0,4.42,544.0,2538.0,Imajica II:The Reconciliation,9780061094156 The magical tale of ill-fated lo...
1450,9780345431912,034543191X,The King of Elfland's Daughter,Edward John Moreton Drax Plunkett Baron Dunsany,Fiction,http://books.google.com/books/content?id=LIoBA...,A young prince ventures into a mysterious fore...,1999.0,3.84,240.0,4718.0,The King of Elfland's Daughter,9780345431912 A young prince ventures into a m...
1748,9780380789023,0380789027,Smoke and Mirrors,Neil Gaiman,Fiction,http://books.google.com/books/content?id=VsEtB...,"In the deft hands of Neil Gaiman, magic is no ...",2005.0,4.05,365.0,50702.0,Smoke and Mirrors:Short Fictions and Illusions,9780380789023 In the deft hands of Neil Gaiman...
2185,9780441010288,0441010288,The Lost Years of Merlin,T. A. Barron,Fiction,http://books.google.com/books/content?id=T5E1z...,"The first title in this bestselling series, no...",2002.0,3.98,304.0,12114.0,The Lost Years of Merlin,9780441010288 The first title in this bestsell...
2601,9780553112443,0553112449,Switch on the Night,Ray Bradbury;Leo Dillon;Diane Dillon,Juvenile Fiction,http://books.google.com/books/content?id=1eUMA...,When a magical little girl named Dark shows a ...,2004.0,4.09,40.0,186.0,Switch on the Night,9780553112443 When a magical little girl named...
2879,9780618894642,0618894640,Children of Húrin,John Ronald Reuel Tolkien,Fiction,http://books.google.com/books/content?id=W6crA...,A fantasy adventure saga set in the early days...,2007.0,3.97,313.0,2612.0,Children of Húrin,9780618894642 A fantasy adventure saga set in ...
3030,9780679756767,0679756760,When I was Puerto Rican,Esmeralda Santiago,Biography & Autobiography,http://books.google.com/books/content?id=OK8ZV...,"Magic, sexual tension, high comedy, and intens...",1994.0,4.01,274.0,8289.0,When I was Puerto Rican,"9780679756767 Magic, sexual tension, high come..."
3700,9780786931934,0786931930,The Search for Power,Margaret Weis,Fiction,http://books.google.com/books/content?id=zdriH...,An anthology of short fiction captures the wor...,2004.0,3.97,335.0,287.0,The Search for Power:Dragons of the War of Souls,9780786931934 An anthology of short fiction ca...
3733,9780802135780,0802135781,Sexing the Cherry,Jeanette Winterson,Fiction,http://books.google.com/books/content?id=P4hcA...,"Interweaving elements of fantasy and history, ...",1989.0,3.83,167.0,11567.0,Sexing the Cherry,9780802135780 Interweaving elements of fantasy...
3879,9780812551235,0812551230,Drum Calls,Jo Clayton,Fiction,http://books.google.com/books/content?id=tm9IG...,"Once every generation, the two worlds of Cland...",1998.0,3.68,341.0,28.0,Drum Calls,"9780812551235 Once every generation, the two w..."
